# Защита персональных данных клиентов

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Умножение-матриц" data-toc-modified-id="Умножение-матриц-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Умножение матриц</a></span></li><li><span><a href="#Алгоритм-преобразования" data-toc-modified-id="Алгоритм-преобразования-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Алгоритм преобразования</a></span></li><li><span><a href="#Проверка-алгоритма" data-toc-modified-id="Проверка-алгоритма-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Проверка алгоритма</a></span><ul class="toc-item"><li><span><a href="#Исходные-данные" data-toc-modified-id="Исходные-данные-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Исходные данные</a></span></li><li><span><a href="#Применение-алгоритма" data-toc-modified-id="Применение-алгоритма-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Применение алгоритма</a></span></li></ul></li><li><span><a href="#Вывод" data-toc-modified-id="Вывод-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Вывод</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

Необходимо защитить данные клиентов страховой компании «Хоть потоп». 
Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось.

- Признаки: пол, возраст и зарплата застрахованного, количество членов его семьи.
- Целевой признак: количество страховых выплат клиенту за последние 5 лет.


## Загрузка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from numpy.linalg import inv

In [2]:
data = pd.read_csv('https://code.s3.yandex.net/datasets/insurance.csv')
data.head()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41.0,49600.0,1,0
1,0,46.0,38000.0,1,1
2,0,29.0,21000.0,0,0
3,0,21.0,41700.0,2,0
4,1,28.0,26100.0,0,0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [4]:
data.describe()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
count,5000.000000,5000.000000,5000.000000,5000.000000,5000.000000
mean,0.499000,30.952800,39916.360000,1.194200,0.148000
std,0.500049,8.440807,9900.083569,1.091387,0.463183
min,0.000000,18.000000,5300.000000,0.000000,0.000000
25%,0.000000,24.000000,33300.000000,0.000000,0.000000
50%,0.000000,30.000000,40200.000000,1.000000,0.000000
75%,1.000000,37.000000,46600.000000,2.000000,0.000000
max,1.000000,65.000000,79000.000000,6.000000,5.000000


Похоже, что с данными все в порядке, никаких явных недочетов нет.

Раз уж мы все-равно собираемся зашифровать данные, то и колонки переименуем.

In [5]:
data = data.rename(columns={'Пол':'sex', 
         'Возраст':'age',
         'Зарплата':'salary',
         'Члены семьи':'family',
         'Страховые выплаты':'payments'})

In [6]:
data.columns

Index(['sex', 'age', 'salary', 'family', 'payments'], dtype='object')

## Умножение матриц

В этом задании вы можете записывать формулы в *Jupyter Notebook.*

Чтобы записать формулу внутри текста, окружите её символами доллара \\$; если снаружи —  двойными символами \\$\\$. Эти формулы записываются на языке вёрстки *LaTeX.* 

Для примера мы записали формулы линейной регрессии. Можете их скопировать и отредактировать, чтобы решить задачу.

Работать в *LaTeX* необязательно.

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$
w = (X^T X)^{-1} X^T y
$$

## Алгоритм преобразования

**Алгоритм**

Сгенерируем случайную матрицу с размером равным количеству признаков и умножим признаки на эту матрицу, и будем считать линейную регрессию по ней $$X'= XB$$

Транспонироване произведения матриц равно произведению транспонированыых матриц в обратном порядке.

$$
w' = (X'^T X')^{-1} X'^T y = ((XB)^T (XB))^{-1} (XB)^T y = (B^T (X^T X) B)^{-1} B^T X^T y = B^{-1} (X^T X)^{-1} (B^T)^{-1} B^T X^T y = B^{-1} w
$$
Тут все матрицы квадратные.
Тогда прогноз посчитается по формуле.
$$
a' = X'w' = XB B^{-1} w = Xw = a
$$

## Проверка алгоритма

### Исходные данные

In [7]:
X = data[['sex', 'age', 'salary', 'family']]
y = data['payments']

Сначала посмотрим R2 на всей выборке. У линейной регрессии по умолчанию score R2.

In [8]:
reg = LinearRegression()
reg.fit(X,y)
reg.score(X, y)

0.4249455028666801

Попробуем кроссвалидацию.

In [9]:
cross_val_score(reg, X, y, cv=5)

array([0.40104201, 0.44663077, 0.41583058, 0.41470783, 0.43735766])

### Применение алгоритма

Сгенирируем случайную матрицу.

In [10]:
B = np.random.RandomState(12345).rand(X.shape[1],X.shape[1])
B

array([[0.92961609, 0.31637555, 0.18391881, 0.20456028],
       [0.56772503, 0.5955447 , 0.96451452, 0.6531771 ],
       [0.74890664, 0.65356987, 0.74771481, 0.96130674],
       [0.0083883 , 0.10644438, 0.29870371, 0.65641118]])

Проверим, что у неё есть обратная матрица.

In [11]:
inv(B)

array([[ 2.1336149 ,  0.5051926 , -1.71113385,  1.33832486],
       [-3.76534462, -2.61352053,  6.72316006, -6.07193701],
       [ 0.97426298,  2.9928115 , -3.49875049,  1.84220045],
       [ 0.13998298, -0.94454066,  0.52375812,  1.652661  ]])

In [12]:
(inv(B) @ B).round()

array([[ 1.,  0.,  0.,  0.],
       [-0.,  1., -0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0., -0., -0.,  1.]])

Почти единичная.

In [13]:
X_new = X @ B

In [14]:
reg = LinearRegression()
reg.fit(X_new,y)
reg.score(X_new, y)

0.42494550286673527

In [15]:
cross_val_score(reg, X_new, y, cv=5)

array([0.40104201, 0.44663077, 0.41583058, 0.41470783, 0.43735766])

## Вывод

- Данные были уже хорошо подготовленны, чистить их не пришломсь.
- В ходе работы было показано, что если домножить признаки на случайную обращаемую матрицу, то точность прогноза линейной регрессии от этого не изменится.
- Было показано на практике, что при домножении признаков на случайную матрицу $B$,точность прогноза практически не меняется.